<a href="https://colab.research.google.com/github/vikpy/DLSem3/blob/master/Assignment_6_hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyforest 
from pyforest import * 

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=a26d1ed14d166b9cc7a6571dd60dec563d2189b351774b7fc0bd0c497356da85
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest



Some methods of hyperparameter tuning:

1.Batch size

2.Learning rate decay

3.Batch norm

4.Adam, RMSProp, Momentum

5.Normalize features

6.Use L1, L2 regularization, Dropout, Data augmentation, Early stopping


#Assignment 6: 

Use __Concepts 1-6__ to improve accuracy on CIFAR100 to +90%

Ans:

References:
1. https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
2. https://docs.google.com/document/d/18JYvbD4L5NhQ8RYSq1xbLVL0bTISk8-0PYF8L3QQ7SY/edit


In [2]:
import keras
from keras.datasets import cifar100
from keras.models import Sequential                         
from keras.layers import Dense, Dropout, Activation, Flatten,Conv2D,MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD, Adam, RMSprop
from matplotlib import pyplot


# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar100.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [3]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.2))
	model.add(Dense(100, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])	
	return model

In [4]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [5]:
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=1)
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

In [ ]:
run_test_harness()

169009152/169001437 [==============================] - 11s 0us/step
Epoch 1/100
782/782 [==============================] - 15s 19ms/step - loss: 4.6016 - accuracy: 0.0144 - val_loss: 4.5607 - val_accuracy: 0.0282
Epoch 2/100
782/782 [==============================] - 14s 18ms/step - loss: 4.4474 - accuracy: 0.0313 - val_loss: 4.3038 - val_accuracy: 0.0573
Epoch 3/100
782/782 [==============================] - 14s 18ms/step - loss: 4.2439 - accuracy: 0.0525 - val_loss: 4.1046 - val_accuracy: 0.0865
Epoch 4/100
782/782 [==============================] - 14s 18ms/step - loss: 4.0652 - accuracy: 0.0773 - val_loss: 3.9426 - val_accuracy: 0.1183
Epoch 5/100
782/782 [==============================] - 14s 18ms/step - loss: 3.9116 - accuracy: 0.1011 - val_loss: 3.7449 - val_accuracy: 0.1491
Epoch 6/100
782/782 [==============================] - 14s 18ms/step - loss: 3.7665 - accuracy: 0.1218 - val_loss: 3.6269 - val_accuracy: 0.1646
Epoch 7/100
782/782 [==============================] - 14s 18m